# plan
- define province names
- create entrypoint for main flow
- create subtasks
  - get api results and parse to dataframe
  - write the dataframe to lakefs


In [1]:
import requests
import pandas as pd
from datetime import datetime
import pytz
from prefect import flow, task # Prefect flow and task decorators

@task
def get_weather_data(province_context={'province':None, 'lat':None, 'lon':None}):
    # API endpoint and parameters
    WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    API_KEY = "a35eabb640026f7b88461e8d99ca6532"  # Replace with your actual API key
    province=province_context['province']
    
    params = {
        "lat": province_context['lat'],
        "lon": province_context['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)


        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict

    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [2]:
provinces = {
    "Pathum Thani":{
        "lat": 14.0134,
        "lon": 100.5304
    },
    "Bangkok":{
            "lat": 13.7367,
            "lon": 100.5232
    },
    "Chiang Mai":{
        "lat": 18.7883,
        "lon": 98.9853
    },
    "Phuket":{
        "lat": 7.9519,
        "lon": 98.3381
    }
}
province='Pathum Thani'
province_context={
    'province':province,
    'lat':provinces[province]['lat'],
    'lon':provinces[province]['lon'],
}
get_weather_data(province_context)

17:15:50.677 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8074
See https://docs.prefect.io/3.0/manage/self-host#self-host-a-prefect-server for more information on running a dedicated Prefect server.

17:15:55.015 | INFO    | Task run 'get_weather_data' - Finished in state Completed()

{'timestamp': datetime.datetime(2025, 5, 7, 17, 15, 55, 13008),
 'year': 2025,
 'month': 5,
 'day': 7,
 'hour': 17,
 'minute': 15,
 'created_at': datetime.datetime(2025, 5, 7, 17, 15, 54, 972562, tzinfo=<DstTzInfo 'Asia/Bangkok' LMT+6:42:00 STD>),
 'requested_province': 'Pathum Thani',
 'location': 'Pathum Thani',
 'weather_main': 'Clouds',
 'weather_description': 'scattered clouds',
 'main.temp': 33.36}

In [ ]:
@flow(name="main-flow", log_prints=True)
def main_flow():
    provinces = {
    "Pathum Thani":{
        "lat": 14.0134,
        "lon": 100.5304
    },
    "Bangkok":{
            "lat": 13.7367,
            "lon": 100.5232
    },
    "Chiang Mai":{
        "lat": 18.7883,
        "lon": 98.9853
    },
    "Phuket":{
        "lat": 7.9519,
        "lon": 98.3381
    }
}
    # for province in provinces.keys:
    #     province_context={
    #         'province':province,
    #         'lat':provinces[province]['lat'],
    #         'lon':provinces[province]['lon'],
    #     }
    #     get_weather_data(province_context)
        
    df=pd.DataFrame([get_weather_data(
        {
            'province':province,
            'lat':provinces[province]['lat'],
            'lon':provinces[province]['lon'],
        }
    ) for province in list(provinces.keys())])
    
        # lakeFS credentials from your docker-compose.yml
    ACCESS_KEY = "access_key"
    SECRET_KEY = "secret_key"
    
    # lakeFS endpoint (running locally)
    lakefs_endpoint = "http://lakefs-dev:8000/"
    
    # lakeFS repository, branch, and file path
    repo = "weather"
    branch = "main"
    path = "weather.parquet"
    
    # Construct the full lakeFS S3-compatible path
    lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"
    
    # Configure storage_options for lakeFS (S3-compatible)
    storage_options = {
        "key": ACCESS_KEY,
        "secret": SECRET_KEY,
        "client_kwargs": {
            "endpoint_url": lakefs_endpoint
        }
    }
    df.to_parquet(
        lakefs_s3_path,
        storage_options=storage_options,
        partition_cols=['year','month','day','hour'],
        
    )
main_flow()

17:17:15.744 | INFO    | Flow run 'gray-bull' - Beginning flow run 'gray-bull' for flow 'main-flow'

17:17:20.582 | INFO    | Task run 'get_weather_data-688' - Finished in state Completed()

17:17:21.089 | INFO    | Task run 'get_weather_data-293' - Finished in state Completed()

17:17:22.602 | INFO    | Task run 'get_weather_data-05b' - Finished in state Completed()

17:17:23.800 | INFO    | Task run 'get_weather_data-95e' - Finished in state Completed()

17:18:35.992 | ERROR   | Flow run 'gray-bull' - Encountered exception during execution: EndpointConnectionError('Could not connect to the endpoint URL: "http://lakefs-dev:8000/weather"')
Traceback (most recent call last):
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 1357, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 995, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 1026, in _resolve_host_with_throttle
    addrs = await self._resolver.resolve(host, port, family=self._family)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\resolver.py", line 39, in resolve
    infos = await self._loop.getaddrinfo(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 933, in getaddrinfo
    return await self.run_in_executor(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
        None, getaddr_func, host, port, family, type, proto, flags)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\concurrent\futures\thread.py", line 59, in run
    result = self.fn(*self.args, **self.kwargs)
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\socket.py", line 977, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiobotocore\httpsession.py", line 222, in send
    response = await session.request(
               ^^^^^^^^^^^^^^^^^^^^^^
    ...<7 lines>...
    )
    ^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\client.py", line 703, in _request
    conn = await self._connector.connect(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        req, traces=traces, timeout=real_timeout
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 548, in connect
    proto = await self._create_connection(req, traces, timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 1056, in _create_connection
    _, proto = await self._create_direct_connection(req, traces, timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\aiohttp\connector.py", line 1363, in _create_direct_connection
    raise ClientConnectorDNSError(req.connection_key, exc) from exc
aiohttp.client_exceptions.ClientConnectorDNSError: Cannot connect to host lakefs-dev:8000 ssl:default [getaddrinfo failed]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\prefect\flow_engine.py", line 763, in run_context
    yield self
  File "c:\Users\natth\AppData\Local\Programs\Python\Python313\Lib\site-packages\prefect\flow_engine.py", line 1370, in run_flow_sync
    engine.call_flow_fn

17:18:36.015 | INFO    | Flow run 'gray-bull' - Finished in state Failed('Flow run encountered an exception: EndpointConnectionError: Could not connect to the endpoint URL: "http://lakefs-dev:8000/weather"')

EndpointConnectionError: Could not connect to the endpoint URL: "http://lakefs-dev:8000/weather"